In [1]:
import open3d as o3d
import numpy as np

# 1. 读取 PLY 文件
pcd = o3d.io.read_point_cloud("dense.ply")

# 2. 如果点云没有法向量，先估计法向量
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
)
# 也可对法向量进行方向统一
pcd.orient_normals_consistent_tangent_plane(100)

# 3. 利用泊松重建生成三角网格
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8)
# 可选：根据密度过滤掉低密度区域（有助于去除噪点和修剪边缘）
density_threshold = np.quantile(densities, 0.2)
vertices_to_remove = densities < density_threshold
mesh.remove_vertices_by_mask(vertices_to_remove)

# 4. 保存为 OBJ 文件（Unity 能直接导入 OBJ 格式）
o3d.io.write_triangle_mesh("output.obj", mesh)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


True

In [2]:
# docker run -it --env DISPLAY=192.168.0.230:0.0  --privileged --runtime nvidia --gpus all --volume D:/sam_docker:/workspace --workdir /workspace --name sam_ubuntu my_sam /bin/bash
